In [1]:
cd ../../../src

/Users/cock/kDrive/PhD/projects/labs/redacted-groupies/src


In [2]:
import yaml
import pickle
import argparse
import numpy as np
from collections import Counter

from pipelines.feature_pipeline import FeaturePipeline

from plotters.data_report import DataReport
from plotters.classification_report import ClassificationReport
from pipelines.crossvalidation_pipeline import CrossValMaker

from matplotlib import pyplot as plt
import seaborn as sns

2025-02-04 01:35:06.873305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
with open('./configs/plot_config.yaml', 'r') as f:
    settings = yaml.load(f, Loader=yaml.FullLoader)
settings['crossvalidation'] = {
    'scorer': {
        'scoring_metrics': settings['classification']['metrics'],
        'fairness_metrics': settings['fairness']['metrics']
    }
}
pipeline = FeaturePipeline(settings)
features, labels, demographics, settings = pipeline.load_sequences()

In [4]:
path = '/Users/cock/kDrive/PhD/projects/labs/redacted-groupies/experiments/baselines/portugal-math/nested_xval_mgarf_modelseeds105_all_folds.pkl'
with open(path, 'rb') as fp:
    experiment = pickle.load(fp)

In [5]:
for fold in range(10):
    truths = [labels[tidx] for tidx in experiment[fold]['test_index']]
    probas = experiment[fold]['y_proba']
    preds = experiment[fold]['y_pred']

In [6]:
p = [idx for idx in range(len(truths)) if truths[idx] == 1]
n = [idx for idx in range(len(truths)) if truths[idx] == 0]

fp = [fidx for fidx in n if preds[fidx] == 1]
fp_probas = [probas[fidx][1] for fidx in n if preds[fidx] == 1]
fn = [fidx for fidx in p if preds[fidx] == 0]
fn_probas = [probas[fidx][1] for fidx in p if preds[fidx] == 0]

tp_probas = [probas[tidx][1] for tidx in p if preds[tidx] == 1]
tn_probas = [probas[tidx][1] for tidx in n if preds[tidx] == 0]

In [7]:
for fold in range(10):
    print('f {}: n est - {}, max depth - {}, population size - {}, generations - {}'.format(
        fold, experiment[fold]['best_params']['n_estimators'], experiment[fold]['best_params']['max_depth'],
        experiment[fold]['best_params']['population_size'], experiment[fold]['best_params']['generations']
    ))

f 0: n est - 7, max depth - 3, population size - 10, generations - 5
f 1: n est - 3, max depth - 7, population size - 20, generations - 10
f 2: n est - 3, max depth - 3, population size - 10, generations - 10
f 3: n est - 7, max depth - 3, population size - 10, generations - 5
f 4: n est - 3, max depth - 3, population size - 10, generations - 10
f 5: n est - 7, max depth - 3, population size - 10, generations - 5
f 6: n est - 7, max depth - 3, population size - 10, generations - 10
f 7: n est - 7, max depth - 7, population size - 20, generations - 10
f 8: n est - 7, max depth - 7, population size - 10, generations - 5
f 9: n est - 7, max depth - 3, population size - 10, generations - 5


In [ ]:
7 - 3 - 10 - 5